<a href="https://colab.research.google.com/github/apeforest/jaxGpt/blob/main/gpt_dev_jax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is a JAX version of the NanoGPT example from Andrej Karpathy's tutorial [LET'S BUILD GPU FROM SCRATCH IN CODE SPELLED OUT.](https://www.youtube.com/watch?v=kCc8FmEb1nY)

* PyTorch version of the [notebook](https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing)

* PyTorch [code](https://github.com/karpathy/*nanoGPT*)

* To learn more about [JAX](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html)

* This note book also uses the following neural network libraries built on top of JAX:
  * [Flax](https://flax.readthedocs.io/en/latest/getting_started.html): a Python neural network library.
  * [Optax](https://optax.readthedocs.io/en/latest/api.html): Commonly used optimizers.



In [29]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-06-21 07:59:11--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2023-06-21 07:59:11 (31.3 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [30]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [31]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [32]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [33]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [34]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


## This section below is implemented using JAX. Please compare with the original PyTorch [notebook](https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing) to see the differences.

In [35]:
# let's now encode the entire text dataset and store it into a jnp array
import jax.numpy as jnp
import jax

jax.config.update("jax_enable_x64", True)

data = jnp.array(encode(text), dtype=jnp.int64)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

(1115394,) int64
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1 39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39
 58 46 43 56  1 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47
 57 46 12  0  0 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53
 50 60 43 42  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47
 56 57 58  6  1 63 53 59  1 49 52 53 61  1 15 39 47 59 57  1 25 39 56 41
 47 59 57  1 47 57  1 41 46 47 43 44  1 43 52 43 51 63  1 58 53  1 58 46
 43  1 54 43 53 54 50 43  8  0  0 13 50 50 10  0 35 43  1 49 52 53 61  5
 58  6  1 61 43  1 49 52 53 61  5 58  8  0  0 18 47 56 57 58  1 15 47 58
 47 64 43 52 10  0 24 43 58  1 59 57  1 49 47 50 50  1 46 47 51  6  1 39
 52 42  1 61 43  5 50 50  1 46 39 

In [36]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [37]:
block_size = 8 # the number of tokens the model looks at, aka context length
train_data[:block_size+1]

Array([18, 47, 56, 57, 58,  1, 15, 47, 58], dtype=int64)

In [38]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is [18] the target: 47
when input is [18 47] the target: 56
when input is [18 47 56] the target: 57
when input is [18 47 56 57] the target: 58
when input is [18 47 56 57 58] the target: 1
when input is [18 47 56 57 58  1] the target: 15
when input is [18 47 56 57 58  1 15] the target: 47
when input is [18 47 56 57 58  1 15 47] the target: 58


In [39]:
from jax import random

# NOTE: JAX does not keep a global ramdom seed
# Users need to pass the random seed key to every call that requires random
# seed.

init_rng = random.PRNGKey(42)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split, rng):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = random.randint(rng, (batch_size, ), 0, len(data) - block_size)
    x = jnp.stack([data[i:i+block_size] for i in ix])
    y = jnp.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train', init_rng)
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
(4, 8)
[[ 0  5 18 53 56 43  1 63]
 [ 1 46 47 51  0 32 53  1]
 [ 1 46 43 56  0 33 52 58]
 [36 17 32 17 30 10  0 20]]
targets:
(4, 8)
[[ 5 18 53 56 43  1 63 53]
 [46 47 51  0 32 53  1 46]
 [46 43 56  0 33 52 58 47]
 [17 32 17 30 10  0 20 43]]
----
when input is [0] the target: 5
when input is [0, 5] the target: 18
when input is [0, 5, 18] the target: 53
when input is [0, 5, 18, 53] the target: 56
when input is [0, 5, 18, 53, 56] the target: 43
when input is [0, 5, 18, 53, 56, 43] the target: 1
when input is [0, 5, 18, 53, 56, 43, 1] the target: 63
when input is [0, 5, 18, 53, 56, 43, 1, 63] the target: 53
when input is [1] the target: 46
when input is [1, 46] the target: 47
when input is [1, 46, 47] the target: 51
when input is [1, 46, 47, 51] the target: 0
when input is [1, 46, 47, 51, 0] the target: 32
when input is [1, 46, 47, 51, 0, 32] the target: 53
when input is [1, 46, 47, 51, 0, 32, 53] the target: 1
when input is [1, 46, 47, 51, 0, 32, 53, 1] the target: 46
when input i

In [40]:
print(xb) # our input to the transformer

[[ 0  5 18 53 56 43  1 63]
 [ 1 46 47 51  0 32 53  1]
 [ 1 46 43 56  0 33 52 58]
 [36 17 32 17 30 10  0 20]]


## For simplicity, start with a Bigram language model

In [41]:
import flax.linen as nn

class BigramLanguageModel(nn.Module):
    vocab_size: int

    # This is explicit setup similar to PyTorch
    def setup(self):
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embed(self.vocab_size, self.vocab_size)

    def __call__(self, idx):

        # idx are (B, T) tensor of integers: B is batch size, T is context length
        # C is channel length which equals to vocab_size here.
        logits = self.token_embedding_table(idx) # (B,T,C)

        return logits

bgm = BigramLanguageModel(vocab_size)
print(bgm.tabulate(jax.random.PRNGKey(0), jnp.ones((vocab_size, vocab_size), dtype=jnp.int64)))


                          BigramLanguageModel Summary                           
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ path          ┃ module        ┃ inputs       ┃ outputs       ┃ params        ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│               │ BigramLangua… │ int64[65,65] │ float32[65,6… │               │
├───────────────┼───────────────┼──────────────┼───────────────┼───────────────┤
│ token_embedd… │ Embed         │ int64[65,65] │ float32[65,6… │ embedding:    │
│               │               │              │               │ float32[65,6… │
│               │               │              │               │               │
│               │               │              │               │ 4,225 (16.9   │
│               │               │              │               │ KB)           │
├───────────────┼───────────────┼──────────────┼───────────────┼───────────────┤
│               │          

In [42]:
from flax.core.frozen_dict import V
import optax

_, rng = random.split(init_rng)
m = BigramLanguageModel(vocab_size)

# Initialize model weights
# NOTE: in JAX, the model doesn't store any weights because JAX is a functional
# programming language and each function call is pure.
params = m.init(rng, jnp.zeros((vocab_size, vocab_size), dtype=jnp.int64))

# Check the parameters
jax.tree_map(lambda x: x.shape, params)

# NOTE: in FLAX, users cannot directly call the model as m(x). Instead, we need
# to explicitly pass in the arguments.
logits = m.apply(params, xb)

print(f'{logits.shape=}')
print(f'{yb.shape=}')

loss = optax.softmax_cross_entropy_with_integer_labels(logits, yb).mean()
print(f'{loss=}')

logits.shape=(4, 8, 65)
yb.shape=(4, 8)
loss=Array(4.1697493, dtype=float32)


In [43]:
# from flax.core.frozen_dict import V
import optax

# This is a JAX implementation of the PyTorch multinomial function.
def multinomial(rng, input, num_samples, replacement=False):
    # input: Tensor of shape (batch_size, num_categories) containing the probabilities for each category
    # num_samples: The number of samples to draw for each batch
    # replacement: Whether to sample with replacement or not

    batch_size, num_categories = input.shape

    # Transform input to log probabilities
    log_probs = jnp.log(input)

    # Generate random samples
    samples = jax.random.categorical(rng, log_probs, axis=-1)

    # If replacement is False, we need to ensure unique samples within each batch
    if not replacement:
        unique_samples = jnp.unique(samples, axis=-1, return_counts=True)
        repeated_indices = unique_samples[1] > 1
        while jnp.any(repeated_indices):
            # Replace repeated samples with new samples
            new_samples = jax.random.categorical(rng, log_probs, jnp.sum(repeated_indices), axis=-1)
            samples = jnp.where(repeated_indices[:, jnp.newaxis], new_samples, samples)

            unique_samples = jnp.unique(samples, axis=-1, return_counts=True)
            repeated_indices = unique_samples[1] > 1

    return jnp.expand_dims(samples, 1)

# The generate method here is generic, even though the bigram model only looks
# at one token before the current.
def generate(model, rng, params, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
        # get the predictions
        logits = model.apply(params, idx)
        # focus only on the last time step
        logits = logits[:, -1, :] # becomes (B, C)

        # apply softmax to get probabilities
        probs = jax.nn.softmax(logits, axis=-1) # (B, C)
        # sample from the distribution
        _, rng = random.split(rng)

        idx_next = multinomial(rng, probs, num_samples=1) # (B, 1)

        # append sampled index to the running sequence
        idx = jnp.concatenate((idx, idx_next), axis=1) # (B, T+1)
    return idx

_, rng = random.split(rng)

idx = generate(m, rng, params, jnp.zeros((1, 1), dtype=jnp.int64), max_new_tokens=100)
print(idx)
print(decode(generate(m, rng, params, idx = jnp.zeros((1, 1), dtype=jnp.int64), max_new_tokens=100)[0].tolist()))


[[ 0 34 28 35 30 62  3  8 56 60 29  0 57 44 34 25 28  9 31 20 16 61  1  1
  49 21 11 35 47 44 45 35 28  1 56 15  3 43  8 56 54 32 20 51 48 56 18 33
   8 52 43 25 53 28 60 50 40 28 26 28 35 60 37 37 62 63 17 53 37 33  9 12
  25 40 21 43 53 28 18 16 51 37  0 13 35 48 37 20 21 39 45 57 35 43 25 20
  28  5 50  3 15]]

VPWRx$.rvQ
sfVMP3SHDw  kI;WifgWP rC$e.rpTHmjrFU.neMoPvlbPNPWvYYxyEoYU3?MbIeoPFDmY
AWjYHIagsWeMHP'l$C


## Write a training loop

In [44]:
!pip install -q clu

In [45]:
from clu import metrics
from flax.training import train_state  # Useful dataclass to keep train state
from flax import struct                # Flax dataclasses
import optax                           # Common loss functions and optimizers


In [46]:
@struct.dataclass
class Metrics(metrics.Collection):
  accuracy: metrics.Accuracy
  loss: metrics.Average.from_output('loss')

class TrainState(train_state.TrainState):
  metrics: Metrics

def create_train_state(module, rng, learning_rate):
  """Creates an initial `TrainState`."""
  params = module.init(rng, jnp.ones((65, 65), dtype=jnp.int64))['params'] # initialize parameters by passing a template image
  tx = optax.adamw(learning_rate)
  return TrainState.create(
      apply_fn=module.apply, params=params, tx=tx,
      metrics=Metrics.empty())

In [47]:
@jax.jit
def train_step(state, batch):
  """Train for a single step."""
  def loss_fn(params):
    logits = state.apply_fn({'params': params}, batch['x'])
    loss = optax.softmax_cross_entropy_with_integer_labels(
        logits=logits, labels=batch['y'].astype(jnp.int32)).mean()
    return loss
  grad_fn = jax.grad(loss_fn)
  grads = grad_fn(state.params)
  state = state.apply_gradients(grads=grads)
  return state

In [48]:
@jax.jit
def compute_metrics(*, state, batch):
  logits = state.apply_fn({'params': state.params}, batch['x'])
  loss = optax.softmax_cross_entropy_with_integer_labels(
        logits=logits, labels=batch['y'].astype(jnp.int32)).mean()
  metric_updates = state.metrics.single_from_model_output(
    logits=logits, labels=batch['y'].astype(jnp.int32), loss=loss)
  metrics = state.metrics.merge(metric_updates)
  state = state.replace(metrics=metrics)
  return state

## Initialize the TrainState



In [49]:
_, rng = random.split(rng)
learning_rate = 0.001
state = create_train_state(bgm, rng, learning_rate)

In [51]:
# print(batch['x'])
for steps in range(10):

  # sample a batch of data
  _, rng = random.split(rng)
  xb, yb = get_batch('train', rng)
  train_batch = {'x': xb, 'y': yb}

  # evaluate loss
  state = train_step(state, train_batch)
  state = compute_metrics(state=state, batch=train_batch)

  # print the loss
  print(f'loss: {state.metrics.compute}')


loss: <bound method Collection.compute of Metrics(_reduction_counter=_ReductionCounter(value=Array(12, dtype=int32)), accuracy=Accuracy(total=Array(2., dtype=float32), count=Array(352, dtype=int64)), loss=Metric.from_output.<locals>.FromOutput(total=Array(46.11226, dtype=float32), count=Array(11, dtype=int64)))>
loss: <bound method Collection.compute of Metrics(_reduction_counter=_ReductionCounter(value=Array(13, dtype=int32)), accuracy=Accuracy(total=Array(2., dtype=float32), count=Array(384, dtype=int64)), loss=Metric.from_output.<locals>.FromOutput(total=Array(50.30012, dtype=float32), count=Array(12, dtype=int64)))>
loss: <bound method Collection.compute of Metrics(_reduction_counter=_ReductionCounter(value=Array(14, dtype=int32)), accuracy=Accuracy(total=Array(2., dtype=float32), count=Array(416, dtype=int64)), loss=Metric.from_output.<locals>.FromOutput(total=Array(54.463566, dtype=float32), count=Array(13, dtype=int64)))>
loss: <bound method Collection.compute of Metrics(_reduct

##  The mathematical trick in self-attention

In [52]:
# consider the following toy example:
B, T, C = 4, 8, 2 # batch, time, channels
x = random.uniform(rng, shape=(B, T, C))
x.shape

(4, 8, 2)

In [53]:
# version 1: bruteforce nested for-loop
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = jnp.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        # NOTE: directly update the xbow like below will error out
        # xbow[b,t] = jnp.mean(xprev)
        # TypeError: '<class 'jaxlib.xla_extension.ArrayImpl'>' object does not
        # support item assignment. JAX arrays are immutable.
        # Instead of ``x[idx] = y``, use ``x = x.at[idx].set(y)`` or another .at[] method:
        xbow = xbow.at[b, t].set(jnp.mean(xprev, axis=0))

In [54]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
a = jnp.tril(jnp.ones((3, 3)))
a = a / jnp.sum(a, 1, keepdims=True)
b = random.randint(rng, (3, 2), 0, 10).astype(jnp.float32)
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
[[1.         0.         0.        ]
 [0.5        0.5        0.        ]
 [0.33333333 0.33333333 0.33333333]]
--
b=
[[7. 0.]
 [9. 3.]
 [6. 1.]]
--
c=
[[7.         0.        ]
 [8.         1.5       ]
 [7.33333333 1.33333333]]


In [55]:
# version 2: using matrix multiply for a weighted aggregation
wei = jnp.tril(jnp.ones((T, T)))
wei = wei / jnp.sum(wei, 1, keepdims=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
jnp.allclose(xbow, xbow2)

Array(True, dtype=bool)

In [56]:
# version 3: use Softmax
def masked_fill(mask, a, fill):
  return jax.lax.select(mask, a, jax.lax.broadcast(fill, a.shape))

tril = jnp.tril(jnp.ones((T, T)))
wei = jnp.zeros((T, T))
# NOTE: np.where is equilant to torch.masked_fill
wei = jnp.where(tril == 0, float('-inf'), wei)
wei = jax.nn.softmax(wei, axis=-1)
xbow3 = wei @ x
jnp.allclose(xbow, xbow3)


Array(True, dtype=bool)

In [70]:
# version 4: self-attention!

B,T,C = 4,8,32 # batch, time, channels
x = random.uniform(rng, shape=(B,T,C))

krng, qrng = random.split(rng)

# let's see a single Head perform self-attention
head_size = 16
key_layer = nn.Dense(head_size, use_bias=False)
query_layer = nn.Dense(head_size, use_bias=False)
value_layer = nn.Dense(head_size, use_bias=False)

k, _ = key_layer.init_with_output(krng, x)   # (B, T, 16)
q, _ = query_layer.init_with_output(qrng, x) # (B, T, 16)

wei =  q @ jnp.swapaxes(k, -2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = jnp.tril(jnp.ones((T, T)))
wei = jnp.where(tril == 0, float('-inf'), wei)
wei = jax.nn.softmax(wei, axis=-1)

vrng, _ = random.split(qrng)
v, _ = value_layer.init_with_output(vrng, x)
out = wei @ v

out.shape

(4, 8, 16)

In [71]:
wei[0]

Array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.3516585 , 0.6483415 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.25266947, 0.30710639, 0.44022414, 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.13072198, 0.20058656, 0.39212968, 0.27656178, 0.        ,
        0.        , 0.        , 0.        ],
       [0.02939103, 0.05089698, 0.29577893, 0.39374995, 0.23018311,
        0.        , 0.        , 0.        ],
       [0.07160868, 0.09075699, 0.222758  , 0.29105647, 0.15217608,
        0.17164378, 0.        , 0.        ],
       [0.0637862 , 0.12791049, 0.41189133, 0.16572621, 0.04846995,
        0.12028385, 0.06193197, 0.        ],
       [0.05714426, 0.07514107, 0.31398966, 0.15248134, 0.14731544,
        0.09157707, 0.09436963, 0.06798153]], dtype=float64)

Notes:

Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
Each example across batch dimension is of course processed completely independently and never "talk" to each other
In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
"self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
"Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below